In [11]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# import getpass
# import os
# import openai


# import dotenv

# dotenv.load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [13]:
from langchain.globals import set_verbose

set_verbose(True)

In [14]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from dotenv import load_dotenv
import os
import uuid
import openai
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_community.vectorstores import Qdrant
from langchain.chains import RetrievalQA, ConversationChain
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


In [15]:
# app = Flask(__name__)
# app.config['UPLOAD_FOLDER'] = 'uploads/'
# app.config['DATABASE_FOLDER'] = 'database/'
# ALLOWED_EXTENSIONS = {'pdf', 'csv'}

# if not os.path.exists(app.config['UPLOAD_FOLDER']):
#     os.makedirs(app.config['UPLOAD_FOLDER'])

# if not os.path.exists(app.config['DATABASE_FOLDER']):
#     os.makedirs(app.config['DATABASE_FOLDER'])



In [16]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

ALLOWED_EXTENSIONS = {'pdf', 'csv'}

docs = []

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_file(filepath):
    global docs

    _, file_extension = os.path.splitext(filepath)
    file_extension = file_extension.lower()

    print(f"Processing file: {filepath}")
    
    if file_extension == '.pdf':
        loader = PyPDFLoader(filepath)
    elif file_extension == '.csv':
        loader = CSVLoader(filepath, encoding='utf-8')
    else:
        print(f"Unsupported file type: {filepath}")
        return

    docs.extend(loader.load_and_split())

def load_all_files_from_database():
    global docs
    database_folder = "database"  # 假設你的database文件夾在當前目錄
    for filename in os.listdir(database_folder):
        if allowed_file(filename):
            filepath = os.path.join(database_folder, filename)
            process_file(filepath)
            print(f"Processed file: {filename}")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return 'All files in the database folder have been loaded and processed.', vectorstore


status, vectorstore = load_all_files_from_database()
print(status)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4", request_timeout=60)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


Processing file: database\CNS16190_correct.pdf
Processed file: CNS16190_correct.pdf
Processing file: database\en_303645v020101p.pdf
Processed file: en_303645v020101p.pdf
Processing file: database\ts_103701v010101p.pdf
Processed file: ts_103701v010101p.pdf
Processing file: database\根據測試項目描述與可應對測試項目的設備功能撰寫的question(Test Scenario).csv
Processed file: 根據測試項目描述與可應對測試項目的設備功能撰寫的question(Test Scenario).csv
All files in the database folder have been loaded and processed.


In [17]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


qa_system_prompt = """您是一位了解CNS16190、EN303645和ETSI TS 103 701的有用助手。\
    使用以下內容來回答最後的問題。如果您不知道答案，只需說您不知道，不要試圖編造答案。
    當您被要求生成測試場景時，應該使用三個要素來生成測試場景：
    1. 給定的條款號和描述
    2. 提供的對應於條款描述的功能
    3. 在ETSI TS 103 701中找到對應於條款號和描述的測試場景
    
    Context: {context}
    """
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

chat_history = []

In [19]:
from langchain_core.messages import HumanMessage



question = "摘要說明 CNS16190 消費者物聯網法規"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

CNS16190是中華民國的國家標準，專門針對消費者物聯網（Internet of Things, IoT）的網宇安全基準要求進行規範。這份標準主要規定了消費者IoT裝置的網路安全控制措施，例如無通用的預設通行碼、實作管理脆弱性報告的方式、保持軟體更新、安全儲存敏感性安全參數、安全通訊等。

其中，標準中的26控制措施6.5條款強調，如果消費者的IoT裝置和服務中蒐集遙測資料，應該向消費者提供有關蒐集哪些遙測資料、如何利用、利用對象及利用目的的資訊。

此外，CNS16190也規定了消費者IoT裝置的架構，說明了這些裝置通常是硬體與軟體組件的彙集，具有實體介面或網路介面，並可以直接透過IP連接性連接至LAN，或間接經由閘道器或集線器連接至LAN。該標準還提供了一些測試場景來衡量裝置是否符合CNS16190的要求。
